In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Student Survey - Jan.xlsx to Student Survey - Jan.xlsx


In [ ]:
import pandas as pd

file_path = "/content/Student Survey - Jan.xlsx"  # Adjust if needed
sheet_dict = pd.read_excel(file_path, sheet_name=None)  # Load all sheets into a dictionary

# Access individual sheets
df_affiliations = sheet_dict.get("affiliations")
df_participants = sheet_dict.get("participants")
df_responses = sheet_dict.get("responses")
df_friends = sheet_dict.get("net_0_Friends")
df_influential = sheet_dict.get("net_1_Influential")
df_feedback = sheet_dict.get("net_2_Feedback")
df_more_time = sheet_dict.get("net_3_MoreTime")
df_advice = sheet_dict.get("net_4_Advice")
df_disrespect = sheet_dict.get("net_5_Disrespect")
df_school_activity = sheet_dict.get("net_affiliation_0_SchoolActivit")

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.4 MB/s eta 0:00:00


In [ ]:
import random
import numpy as np
import pandas as pd
import networkx as nx
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt

Encoding relationships to graphs

In [ ]:
# Assuming df_friends is already loaded
# Remove self-loops (where Source == Target)
df_friends = df_friends[df_friends["Source"] != df_friends["Target"]]

# Create a directed graph (DiGraph)
G = nx.DiGraph()
for _, row in df_friends.iterrows():
    G.add_edge(row["Source"], row["Target"])

# Create a dictionary to store friendships
friendship_dict = {node: list(G.neighbors(node)) for node in G.nodes()}

# Number of students and classes (predefined)
num_students = df_participants['Participant-ID'].nunique()
num_classes = 6

In [ ]:
# Assuming df_influential is already loaded
# Remove self-loops (where Source == Target)
df_influential = df_influential[df_influential["Source"] != df_influential["Target"]]

# Create a directed graph (DiGraph)
G_influence = nx.DiGraph()
for _, row in df_influential.iterrows():
    G_influence.add_edge(row["Source"], row["Target"])

# Create a dictionary to store influential relationships
influence_dict = {node: list(G_influence.neighbors(node)) for node in G_influence.nodes()}

In [ ]:
# Assuming df_feedback is already loaded
# Remove self-loops (where Source == Target)
df_feedback = df_feedback[df_feedback["Source"] != df_feedback["Target"]]

# Create a directed graph (DiGraph)
G_feedback = nx.DiGraph()
for _, row in df_feedback.iterrows():
    G_feedback.add_edge(row["Source"], row["Target"])

# Create a dictionary to store feedback relationships
feedback_dict = {node: list(G_feedback.neighbors(node)) for node in G_feedback.nodes()}

In [ ]:
# Assuming df_more_time is already loaded
# Remove self-loops (where Source == Target)
df_more_time = df_more_time[df_more_time["Source"] != df_more_time["Target"]]

# Create a directed graph (DiGraph)
G_more_time = nx.DiGraph()
for _, row in df_more_time.iterrows():
    G_more_time.add_edge(row["Source"], row["Target"])

# Create a dictionary to store "more time" relationships
more_time_dict = {node: list(G_more_time.neighbors(node)) for node in G_more_time.nodes()}

In [ ]:
# Assuming df_advice is already loaded
# Remove self-loops (where Source == Target)
df_advice = df_advice[df_advice["Source"] != df_advice["Target"]]

# Create a directed graph (DiGraph)
G_advice = nx.DiGraph()
for _, row in df_advice.iterrows():
    G_advice.add_edge(row["Source"], row["Target"])

# Create a dictionary to store advice relationships
advice_dict = {node: list(G_advice.neighbors(node)) for node in G_advice.nodes()}

In [ ]:
# Assuming df_disrespect is already loaded
# Remove self-loops (where Source == Target)
df_disrespect = df_disrespect[df_disrespect["Source"] != df_disrespect["Target"]]

# Create a directed graph (DiGraph)
G_disrespect = nx.DiGraph()
for _, row in df_disrespect.iterrows():
    G_disrespect.add_edge(row["Source"], row["Target"])

# Create a dictionary to store disrespect relationships
disrespect_dict = {node: list(G_disrespect.neighbors(node)) for node in G_disrespect.nodes()}

CP-SAT

In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [ ]:
from ortools.sat.python import cp_model
import pandas as pd

# Assuming df_participants is already loaded with student data
student_ids = df_participants['Participant-ID'].tolist()

# Create a model instance
model = cp_model.CpModel()

# Define variables
num_students = len(student_ids)
num_classes = 6
students = student_ids
classes = range(num_classes)

# Class assignments for each student
class_assignments = {student_id: model.NewIntVar(0, num_classes - 1, f'class_{student_id}') for student_id in students}

# Define class sizes (strict class sizes)
class_capacities = [29, 29, 29, 30, 29, 29]  # Class capacities for each class
balance_threshold = 1  # Increased allowed difference between classes (+/- 3 students)

# Add "one student per class" constraint
for student_id in students:
    model.Add(class_assignments[student_id] >= 0)
    model.Add(class_assignments[student_id] < num_classes)  # Ensure the class assignment is valid

# Create strict class size constraints with a relaxed balance threshold
for c in classes:
    bool_vars = [model.NewBoolVar(f'student_{student_id}_in_class_{c}') for student_id in students]
    for student_id in students:
        model.Add(class_assignments[student_id] == c).OnlyEnforceIf(bool_vars[students.index(student_id)])
        model.Add(class_assignments[student_id] != c).OnlyEnforceIf(bool_vars[students.index(student_id)].Not())

    # Relax class size constraint further
    model.Add(sum(bool_vars) >= class_capacities[c] - balance_threshold)
    model.Add(sum(bool_vars) <= class_capacities[c] + balance_threshold)

# Solve with solution limits
solver = cp_model.CpSolver()

# Set up a solution counter to limit the number of solutions to 10 for faster feedback
max_solutions = 10
solutions = []

# Track previously found solutions to avoid duplicates
found_solutions = set()

# Set a max time for the solver (e.g., 30 seconds)
solver.parameters.max_time_in_seconds = 30

status = solver.Solve(model)

while status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # Extract the solution as a tuple (this will ensure it's hashable for set operations)
    solution = tuple(solver.Value(class_assignments[student_id]) for student_id in students)

    # Ensure the solution is not a duplicate
    if solution not in found_solutions:
        solutions.append(solution)
        found_solutions.add(solution)

    # Stop once 10 solutions are found
    if len(solutions) >= max_solutions:
        break

    # Solve for the next solution
    status = solver.Solve(model)

# Print all the solutions
for solution in solutions:
    print(dict(zip(students, solution)))

# Print total number of solutions
print(f"Total number of allocation options generated: {len(solutions)}")

{32437: 0, 32540: 3, 32443: 5, 32514: 4, 32485: 1, 32436: 1, 32553: 4, 32463: 0, 32444: 3, 32428: 4, 32494: 0, 32484: 2, 32541: 1, 32445: 4, 32470: 3, 32395: 0, 32531: 3, 32440: 3, 32550: 0, 32511: 0, 32556: 2, 32549: 1, 32400: 2, 32505: 2, 32417: 4, 32478: 1, 32477: 2, 32474: 3, 32493: 0, 32413: 3, 32459: 1, 32534: 4, 32527: 4, 32532: 5, 32399: 1, 32409: 2, 32475: 2, 32462: 0, 32498: 4, 32458: 1, 32404: 0, 32517: 0, 32401: 5, 32504: 3, 32448: 0, 32522: 1, 32509: 1, 32397: 5, 32523: 3, 32512: 5, 32455: 4, 32489: 5, 32432: 4, 32476: 0, 32516: 5, 32467: 1, 32408: 3, 32420: 2, 32497: 2, 32515: 2, 32429: 4, 32415: 3, 32435: 3, 32405: 5, 32392: 4, 32495: 3, 32427: 5, 32421: 4, 32422: 4, 32500: 4, 32403: 3, 32557: 3, 32499: 3, 32433: 5, 32402: 5, 32491: 2, 32453: 3, 32530: 2, 32544: 1, 32508: 1, 32407: 5, 32565: 5, 32529: 5, 32457: 2, 32545: 5, 32492: 1, 32406: 4, 32479: 0, 32490: 4, 32513: 5, 32480: 4, 32546: 0, 32535: 4, 32456: 5, 32551: 5, 32446: 3, 32451: 5, 32425: 1, 32471: 0, 32528: 1,

GREEDY ALGO

In [ ]:
import pandas as pd
import random

# Assuming df_participants is already loaded with student data
student_ids = df_participants['Participant-ID'].tolist()
gpas = df_participants['Perc_Academic'].tolist()

# Class sizes (e.g., 29 or 30 students per class)
class_capacities = [29, 29, 29, 30, 29, 29]  # Class capacities for each class
num_classes = len(class_capacities)

# Number of solutions to generate
max_solutions = 100

# Initialize to store solutions
all_solutions = []

# Function to generate one solution
def generate_solution():
    # Shuffle students randomly
    students = student_ids[:]
    random.shuffle(students)  # Shuffle students for randomness

    # Initialize class assignments and class sizes
    class_assignments = {student_id: None for student_id in student_ids}
    class_sizes = [0] * num_classes  # To track number of students assigned to each class

    # Greedy assignment: Assign students to classes
    for student_id in students:
        # Find the class with the least number of students that isn't full
        for c in range(num_classes):
            if class_sizes[c] < class_capacities[c]:
                # Assign student to class c
                class_assignments[student_id] = c
                class_sizes[c] += 1
                break

    return class_assignments

# Generate multiple solutions
while len(all_solutions) < max_solutions:
    solution = generate_solution()

    # Check if this solution is distinct
    if solution not in all_solutions:
        all_solutions.append(solution)

# Output the solutions
for idx, solution in enumerate(all_solutions):
    print(f"Solution {idx + 1}: {solution}")

# Print the total number of solutions generated
print(f"Total number of allocation options generated: {len(all_solutions)}")

Solution 1: {32437: 5, 32540: 3, 32443: 2, 32514: 5, 32485: 0, 32436: 1, 32553: 2, 32463: 0, 32444: 5, 32428: 2, 32494: 4, 32484: 3, 32541: 3, 32445: 3, 32470: 3, 32395: 1, 32531: 5, 32440: 4, 32550: 0, 32511: 5, 32556: 3, 32549: 5, 32400: 5, 32505: 3, 32417: 0, 32478: 2, 32477: 1, 32474: 3, 32493: 3, 32413: 3, 32459: 3, 32534: 0, 32527: 1, 32532: 2, 32399: 2, 32409: 1, 32475: 3, 32462: 5, 32498: 3, 32458: 1, 32404: 5, 32517: 5, 32401: 0, 32504: 1, 32448: 3, 32522: 4, 32509: 0, 32397: 0, 32523: 1, 32512: 4, 32455: 4, 32489: 3, 32432: 4, 32476: 2, 32516: 0, 32467: 1, 32408: 2, 32420: 3, 32497: 0, 32515: 0, 32429: 5, 32415: 3, 32435: 2, 32405: 2, 32392: 5, 32495: 2, 32427: 0, 32421: 1, 32422: 4, 32500: 5, 32403: 3, 32557: 1, 32499: 1, 32433: 0, 32402: 0, 32491: 3, 32453: 2, 32530: 4, 32544: 1, 32508: 3, 32407: 1, 32565: 0, 32529: 1, 32457: 2, 32545: 4, 32492: 3, 32406: 3, 32479: 5, 32490: 2, 32513: 4, 32480: 5, 32546: 4, 32535: 5, 32456: 2, 32551: 2, 32446: 3, 32451: 2, 32425: 5, 32471: 

GA: Fitness

In [ ]:
# Define weights for each objective (adjust these based on your priorities)
weight_friendship = 1.0
weight_influence = 1.0
weight_feedback = 1.0
weight_time = 1.0
weight_advice = 1.0
weight_disrespect = 1.0

# Updated fitness functions that use weights

def calculate_friendship_density(individual):
    class_friendships = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_friendships[class_id].append(student_id)

    total_friendships = 0
    total_possible_friendships = 0

    for class_id, students in class_friendships.items():
        if len(students) < 2:
            continue  # Skip classes with fewer than 2 students
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in friendship_dict.get(student1, []):  # Safe access to friendship_dict
                    total_friendships += 1
                total_possible_friendships += 1

    return total_friendships / total_possible_friendships if total_possible_friendships > 0 else 0

def calculate_influence_balance(individual):
    degree_centrality = nx.degree_centrality(G_influence)
    class_influence = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        if student_id in degree_centrality:
            class_influence[class_id].append(degree_centrality[student_id])

    total_influence = 0
    total_classes = 0
    for class_id, students in class_influence.items():
        if len(students) == 0:
            continue
        class_avg_influence = sum(students) / len(students)
        total_influence += class_avg_influence
        total_classes += 1

    return total_influence / total_classes if total_classes > 0 else 0

def calculate_feedback_exchange(individual):
    class_feedback = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_feedback[class_id].append(student_id)

    total_feedback = 0
    total_possible_feedback = 0
    for class_id, students in class_feedback.items():
        if len(students) < 2:
            continue
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in feedback_dict.get(student1, []):
                    total_feedback += 1
                total_possible_feedback += 1

    return total_feedback / (total_possible_feedback if total_possible_feedback > 0 else 1)

def calculate_time_spent_together(individual):
    class_time = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_time[class_id].append(student_id)

    total_time = 0
    total_possible_time = 0
    for class_id, students in class_time.items():
        if len(students) < 2:
            continue
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in more_time_dict.get(student1, []):
                    total_time += 1
                total_possible_time += 1

    return total_time / (total_possible_time if total_possible_time > 0 else 1)

def calculate_advice_exchange(individual):
    class_advice = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_advice[class_id].append(student_id)

    total_advice = 0
    total_possible_advice = 0
    for class_id, students in class_advice.items():
        if len(students) < 2:
            continue
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in advice_dict.get(student1, []):
                    total_advice += 1
                total_possible_advice += 1

    return total_advice / (total_possible_advice if total_possible_advice > 0 else 1)

def calculate_disrespect(individual):
    class_disrespect = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_disrespect[class_id].append(student_id)

    total_disrespect = 0
    for class_id, students in class_disrespect.items():
        if len(students) < 2:
            continue
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in disrespect_dict.get(student1, []):
                    total_disrespect += 1

    return total_disrespect

# Define the multi-objective fitness function with weighted objectives
def calculate_multi_objective_fitness(individual):
    # Calculate each objective
    friendship_density = calculate_friendship_density(individual)
    influence_balance = calculate_influence_balance(individual)
    feedback_exchange = calculate_feedback_exchange(individual)
    time_spent_together = calculate_time_spent_together(individual)
    advice_exchange = calculate_advice_exchange(individual)
    disrespect = calculate_disrespect(individual)

    # Apply weights to each objective
    weighted_friendship = friendship_density * weight_friendship
    weighted_influence = influence_balance * weight_influence
    weighted_feedback = feedback_exchange * weight_feedback
    weighted_time = time_spent_together * weight_time
    weighted_advice = advice_exchange * weight_advice
    weighted_disrespect = disrespect * weight_disrespect

    # Return the tuple of weighted fitness values
    return weighted_friendship, weighted_influence, weighted_feedback, weighted_time, weighted_advice, weighted_disrespect

GA: running

In [ ]:
# GA setup
creator.create("FitnessMulti", base.Fitness, weights=(1.0, 1.0, 1.0, 1.0, 1.0, -1.0))  # Adjust weights for your problem
creator.create("Individual", dict, fitness=creator.FitnessMulti)

# Convert greedy solutions into GA-compatible individuals
def convert_to_individual(greedy_solution):
    individual = creator.Individual({student_id: greedy_solution[student_id] for student_id in student_ids})
    individual.fitness.values = calculate_multi_objective_fitness(individual)
    return individual

# Use the `all_solutions` list as the initial population for GA
population = [convert_to_individual(solution) for solution in all_solutions]

# Mutation function (randomly change one student's class assignment)
def mutate(individual):
    student_id = random.choice(list(individual.keys()))
    individual[student_id] = random.randint(0, num_classes - 1)  # Randomly assign to a class
    individual.fitness.values = calculate_multi_objective_fitness(individual)
    return individual,

# Crossover function (swap assignments between two individuals)
def crossover(ind1, ind2):
    cxpoint = random.randint(1, len(ind1) - 1)
    keys = list(ind1.keys())
    for i in range(cxpoint, len(keys)):
        key = keys[i]
        ind1[key], ind2[key] = ind2[key], ind1[key]
    ind1.fitness.values = calculate_multi_objective_fitness(ind1)
    ind2.fitness.values = calculate_multi_objective_fitness(ind2)
    return ind1, ind2

# Set up the genetic algorithm toolbox
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, convert_to_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selNSGA2)  # NSGA2 for multi-objective optimization

toolbox.register("evaluate", calculate_multi_objective_fitness)

# Run the genetic algorithm
population_size = 1000
num_generations = 50
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations,
                    stats=None, halloffame=None, verbose=True)

# Get the best individual (based on Pareto dominance)
best_individual = tools.selBest(population, 1)[0]
print(f"Best solution (class assignments): {best_individual}")

gen	nevals
0  	0     
1  	74    
2  	76    
3  	77    
4  	80    
5  	70    
6  	84    
7  	75    
8  	84    
9  	80    
10 	74    
11 	73    
12 	73    
13 	75    
14 	73    
15 	72    
16 	72    
17 	72    
18 	77    
19 	78    
20 	81    
21 	86    
22 	67    
23 	71    
24 	70    
25 	77    
26 	79    
27 	64    
28 	75    
29 	68    
30 	77    
31 	82    
32 	81    
33 	80    
34 	71    
35 	62    
36 	85    
37 	72    
38 	77    
39 	88    
40 	73    
41 	73    
42 	83    
43 	74    
44 	74    
45 	77    
46 	76    
47 	68    
48 	76    
49 	76    
50 	69    
Best solution (class assignments): {32437: 2, 32540: 2, 32443: 5, 32514: 5, 32485: 0, 32436: 0, 32553: 1, 32463: 4, 32444: 4, 32428: 5, 32494: 3, 32484: 2, 32541: 1, 32445: 4, 32470: 4, 32395: 0, 32531: 0, 32440: 1, 32550: 3, 32511: 0, 32556: 2, 32549: 2, 32400: 4, 32505: 4, 32417: 1, 32478: 0, 32477: 1, 32474: 5, 32493: 4, 32413: 3, 32459: 0, 32534: 4, 32527: 5, 32532: 2, 32399: 2, 32409: 3, 32475: 2, 32462: 5, 32498: 4, 32

Evaluation

In [ ]:
# Get the best individual (solution with the highest fitness)
best_individual = tools.selBest(population, 1)[0]

# Evaluate the fitness of the best individual
fitness_score = calculate_multi_objective_fitness(best_individual)

# Print the fitness values for each objective
print("Fitness values for the best individual (class assignments):")
print(f"Friendship density: {fitness_score[0]}")
print(f"Influence balance: {fitness_score[1]}")
print(f"Feedback exchange: {fitness_score[2]}")
print(f"Time spent together: {fitness_score[3]}")
print(f"Advice exchange: {fitness_score[4]}")
print(f"Disrespect: {fitness_score[5]}")

# You can also print the class assignments of the best individual
print(f"Best individual (class assignments): {best_individual}")

Fitness values for the best individual (class assignments):
Friendship density: 0.05096040768326147
Influence balance: 0.03589076621260807
Feedback exchange: 0.00901607212857703
Time spent together: 0.01960015680125441
Advice exchange: 0.013328106624852998
Disrespect: 9.0
Best individual (class assignments): {32437: 2, 32540: 2, 32443: 5, 32514: 5, 32485: 0, 32436: 0, 32553: 1, 32463: 4, 32444: 4, 32428: 5, 32494: 3, 32484: 2, 32541: 1, 32445: 4, 32470: 4, 32395: 0, 32531: 0, 32440: 1, 32550: 3, 32511: 0, 32556: 2, 32549: 2, 32400: 4, 32505: 4, 32417: 1, 32478: 0, 32477: 1, 32474: 5, 32493: 4, 32413: 3, 32459: 0, 32534: 4, 32527: 5, 32532: 2, 32399: 2, 32409: 3, 32475: 2, 32462: 5, 32498: 4, 32458: 4, 32404: 0, 32517: 3, 32401: 1, 32504: 3, 32448: 4, 32522: 4, 32509: 3, 32397: 4, 32523: 2, 32512: 5, 32455: 3, 32489: 3, 32432: 2, 32476: 0, 32516: 2, 32467: 1, 32408: 4, 32420: 5, 32497: 0, 32515: 4, 32429: 5, 32415: 4, 32435: 3, 32405: 4, 32392: 0, 32495: 3, 32427: 5, 32421: 1, 32422: 1,

In [ ]:
# Extract the Pareto front (solutions that are not dominated)
pareto_front = tools.sortNondominated(population, len(population), first_front_only=True)[0]

# Print the Pareto front (solutions that balance the objectives)
print("Pareto front solutions (class assignments and fitness scores):")
for ind in pareto_front:
    print(f"Class assignments: {ind}, Fitness: {ind.fitness.values}")

Pareto front solutions (class assignments and fitness scores):
Class assignments: {32437: 3, 32540: 1, 32443: 2, 32514: 5, 32485: 0, 32436: 2, 32553: 2, 32463: 1, 32444: 3, 32428: 4, 32494: 3, 32484: 5, 32541: 4, 32445: 2, 32470: 5, 32395: 5, 32531: 1, 32440: 1, 32550: 5, 32511: 2, 32556: 1, 32549: 3, 32400: 0, 32505: 5, 32417: 2, 32478: 5, 32477: 5, 32474: 5, 32493: 4, 32413: 2, 32459: 0, 32534: 3, 32527: 5, 32532: 3, 32399: 5, 32409: 2, 32475: 1, 32462: 3, 32498: 1, 32458: 2, 32404: 2, 32517: 1, 32401: 5, 32504: 5, 32448: 4, 32522: 5, 32509: 4, 32397: 3, 32523: 0, 32512: 3, 32455: 2, 32489: 0, 32432: 0, 32476: 5, 32516: 0, 32467: 3, 32408: 1, 32420: 2, 32497: 5, 32515: 2, 32429: 5, 32415: 2, 32435: 1, 32405: 2, 32392: 5, 32495: 5, 32427: 2, 32421: 0, 32422: 4, 32500: 5, 32403: 4, 32557: 3, 32499: 3, 32433: 4, 32402: 3, 32491: 5, 32453: 0, 32530: 3, 32544: 4, 32508: 1, 32407: 4, 32565: 0, 32529: 4, 32457: 2, 32545: 5, 32492: 0, 32406: 1, 32479: 4, 32490: 5, 32513: 1, 32480: 4, 32546: 